In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [ ]:
display_width = 100 # in percentage

from IPython.display import display, HTML
display(HTML("<style>.container { width:"+str(display_width)+"% !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# import sys
# sys.path.append("/Users/sudhanshugupta/Library/Python/3.9/lib/python/site-packages")

In [ ]:
import pandas as pd
import numpy as np

import os

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from EDA import EDA_movie_reviews
from preprocessing import preprocMovieReview
from feature_extraction import textVectorizer
from utils import train_val_test_split
from classification import LogisticRegressionClf, BiLSTMClassifierWrapper

# Read Data

In [ ]:
df_movie_reviews = pd.read_csv("./data/movie_review_train.tsv", delimiter="\t")
df_movie_reviews.sample(10)

## Basic Text Preprocessing

In [ ]:
text_preprocessor = preprocMovieReview(df_movie_reviews['review'])
df_movie_reviews['review_clean'] = text_preprocessor.basic_text_sanitization_pipeline()

# Exploratory Data Analysis

In [ ]:
eda = EDA_movie_reviews(df_movie_reviews, reviews_col='review_clean', target_col='sentiment')
eda.visualize_wordcloud()
eda.visulaize_class_distribution()
eda.statistics_on_review_text()
eda.show_common_n_grams(n=5, show_count=10)

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = train_val_test_split(df_movie_reviews, feature_cols='review_clean', target_col='sentiment', test_percent=20, val_percent=0)
# print(f"{X_train.shape=}, {y_train.shape=}, {X_test.shape=}, {y_test.shape=}, {X_val.shape=}, {y_val.shape=}")

# Establish Baseline: TFIDF Vectorizer + Basic Classifier

In [ ]:
tfidf_vectorizer = textVectorizer(vectorizer_type='tfidf')
X_train = tfidf_vectorizer.apply_transform_train(X_train)
X_test = tfidf_vectorizer.apply_transform_test(X_test)

print(X_train.shape, X_test.shape)

In [ ]:
clf_logistic_regression = LogisticRegressionClf()
clf_logistic_regression.fit_classifier(X_train, y_train)
y_pred = clf_logistic_regression.predict_classifier(X_test)
clf_logistic_regression.evaluate_classifier(y_test, y_pred)

# Improved Classifier: TFIDF Vectorizer + BiLSTM

In [ ]:
# Example usage
input_size = X_train.shape[1]
hidden_size = 16
output_size = 2
num_epochs = 10
batch_size = 8
lr_init = 1e-3

clf_bilstm = BiLSTMClassifierWrapper(input_size, hidden_size, output_size, batch_size, num_epochs, lr_init)
clf_bilstm.fit_classifier(X_train, y_train)
y_pred = clf_bilstm.predict_classifier(X_test)
clf_bilstm.evaluate_classifier(y_test, y_pred)

# Improved Embeddings: Glove + BiLSTM

# Transformer Based: BERT

In [ ]:
from classification import DistilBERTClassifier

In [ ]:
clf_distilbert = DistilBERTClassifier()
clf_distilbert.fit_classifier(X_train, y_train)

# Error Analysis